**Install required libraries**

In [ ]:
# install required libraries
%pip install azure-ai-documentintelligence
%pip install openai azure-kusto-data langchain tenacity langchain-openai pypdf
%pip install -qU langchain_community pypdf
%pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet
%pip install --upgrade certifi

In [ ]:
from openai import AzureOpenAI
from IPython.display import display, HTML
import os
import textwrap
import json
from notebookutils import mssparkutils
from tenacity import retry, wait_random_exponential, stop_after_attempt

# import from kusto
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
#import from langchain
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader


**Get Keys for Azure and Fabric Services from Key Vault**

In [ ]:
AZURE_OPENAI_KEY = mssparkutils.credentials.getSecret('https://<YOUR_AZURE_KEYVAULT>.vault.azure.net/','azureopenaikey')

AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME = 'gpt-4o'
AZURE_OPENAI_GPT4o_ENDPOINT = mssparkutils.credentials.getSecret('https://<YOUR_AZURE_KEYVAULT>.vault.azure.net/','gpt4omodeluri')

AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = 'text-embedding-ada-002'
AZURE_OPENAI_EMBEDDING_ENDPOINT = mssparkutils.credentials.getSecret('https://<YOUR_AZURE_KEYVAULT>.vault.azure.net/','adaembeddingmodeluri')

KUSTO_URI = mssparkutils.credentials.getSecret('https://<YOUR_AZURE_KEYVAULT>.vault.azure.net/','kustoqueryuri')
KUSTO_DATABASE = mssparkutils.credentials.getSecret('https://<YOUR_AZURE_KEYVAULT>.vault.azure.net/','kustodbname')
KUSTO_TABLE = "embeddings1" # table name used in this code, but you can choose any name
KDB_ACCESSTOKEN = mssparkutils.credentials.getToken(KUSTO_URI)


**Create Azure OpenAI client**

In [ ]:
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_GPT4o_ENDPOINT,
    api_key=AZURE_OPENAI_KEY, 
    api_version= "2024-08-01-preview"
    
)




In [ ]:

# create delay
@retry(wait=wait_random_exponential(min=1, max= 20), stop=stop_after_attempt(6))

def generate_embeddings(text):
    # Ensure the text is a string and replace new lines with spaces
    if not isinstance(text, str):
        raise ValueError("Input text must be a string")

    txt = text.replace("\n", " ")

    # Generate embeddings using the client
    try:
        response = client.embeddings.create(input=[txt], model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME)
        embedding = response.data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        raise


**Create embeddings**

In [29]:
# splitting into chunks with overlap

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 20
)

documentname = "report_2024"
filename = "/lakehouse/default/Files/Microsoft-2024-Environmental-Sustainability-Report.pdf"
loader = PyPDFLoader(filename)
pages = loader.load_and_split(text_splitter=splitter)
print("Number of pages: ", len(pages))

StatementMeta(, 256818ca-a4a0-4a3e-9d94-7c6ad228cdfd, 55, Finished, Available, Finished)

Number of pages:  313


In [14]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 20
)

documentname = "report_2022"
filename = "/lakehouse/default/Files/2022-Environmental-Sustainability-Report.pdf"
loader = PyPDFLoader(filename)
pages = loader.load_and_split(text_splitter=splitter)
print("Number of pages: ", len(pages))

StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 32, Finished, Available, Finished)

Number of pages:  280


**Save pages in a dataframe**

In [15]:
import pandas as pd
df = pd.DataFrame(columns=['document_name', 'content', 'embedding'])
for page in pages:
    df.loc[len(df.index)]= [documentname, page.page_content,""]

df.head()

StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 33, Finished, Available, Finished)

,document_name,content,embedding
0,report_2022,2022 \nEnvironmental \nSustainability \nReport...,
1,report_2022,Color Palette\nNames are TBC\nContents\nOvervi...,
2,report_2022,and green skilling \n76\nAppendix \nAppendix A...,
3,report_2022,Overview\nReviewing our 2022 \nprogress and le...,
4,report_2022,Foreword\nEnabling sustainability for our comp...,


**Embeddings using Azure OpenAI embedding model Ada**

In [16]:
df["embedding"] = df.content.apply(lambda x: generate_embeddings(x) )
print(df.head(2))

StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 34, Finished, Available, Finished)

  document_name                                            content  \
0   report_2022  2022 \nEnvironmental \nSustainability \nReport...   
1   report_2022  Color Palette\nNames are TBC\nContents\nOvervi...   

                                           embedding  
0  [-0.0016355628613382578, -0.021236147731542587...  
1  [0.016158754006028175, -0.02216360718011856, -...  


**Write Spark Dataframe into Kusto Table**

In [60]:
from pyspark.sql import SparkSession

# Initialize Spark session with Kusto connector
spark = SparkSession.builder \
    .appName("KustoSparkConnector") \
    .config("spark.jars.packages", "com.microsoft.azure.kusto:spark-kusto-connector:1.0.0") \
    .getOrCreate()

# Ensure df is a valid Pandas DataFrame or collection
try:
    df_spark = spark.createDataFrame(df)
except Exception as e:
    print(f"Error creating Spark DataFrame: {e}")
    raise

# Write DataFrame to Kusto
try:
    df_spark.write \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("kustoCluster", KUSTO_URI) \
        .option("kustoDatabase", KUSTO_DATABASE) \
        .option("kustoTable", KUSTO_TABLE) \
        .option("kustoAuthenticationMode", "AADToken") \
        .option("accessToken", KDB_ACCESSTOKEN) \
        .mode("Append") \
        .save()
    print("DataFrame successfully written to Kusto table.")
except Exception as e:
    print(f"Error writing DataFrame to Kusto: {e}")
    raise


StatementMeta(, 256818ca-a4a0-4a3e-9d94-7c6ad228cdfd, 86, Finished, Available, Finished)

DataFrame successfully written to Kusto table.


**Vector Search with Fabric Eventhouse**

In [17]:
# function to call gpt-4o

def call_azureopenai(text):
    response = client.chat.completions.create(
        model= AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME, 
        messages= text, 
        temperature=0
    )
    return response.choices[0].messages.content


StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 35, Finished, Available, Finished)

In [18]:


#function to retrieve answers from embeddings with similarity search

def get_answer_kustodb(question, answers=1):
    searchedEmbedding = generate_embeddings(question)
    str_searched_embedding = searchedEmbedding #json.dumps(searchedEmbedding)
    kusto_query = f"""
    {KUSTO_TABLE}
    | extend similarity = series_cosine_similarity(dynamic({str_searched_embedding}), embedding)
    | top {answers} by similarity desc
    | project answer_column, similarity
    """
    kustoDF = spark.read \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("kustoCluster", KUSTO_URI) \
        .option("kustoDatabase", KUSTO_DATABASE) \
        .option("kustoQuery", kusto_query) \
        .option("accessToken", KDB_ACCESSTOKEN) \
        .load()

    return kustoDF


StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 36, Finished, Available, Finished)

**Create functions to get answer from KustoDB and to call Azure OpenAI**

In [ ]:
from pyspark.sql import SparkSession
import json

# Initialize Spark session with Kusto connector
spark = SparkSession.builder \
    .appName("KustoSparkConnector") \
    .config("spark.jars.packages", "com.microsoft.azure.kusto:spark-kusto-connector:1.0.0") \
    .getOrCreate()

# Function to query Kusto DB and retrieve answers based on cosine similarity
def get_answer_kustodb(question, answers=1):
    searched_embedding = generate_embeddings(question)
    str_searched_embedding = json.dumps(searched_embedding)
    kusto_query = f"""
    {KUSTO_TABLE}
    | extend similarity = series_cosine_similarity(dynamic({str_searched_embedding}), todynamic(embedding))
    | top {answers} by similarity desc
    | project content, similarity
    """

    try:
        kustoDF = spark.read \
            .format("com.microsoft.kusto.spark.synapse.datasource") \
            .option("kustoCluster", KUSTO_URI) \
            .option("kustoDatabase", KUSTO_DATABASE) \
            .option("kustoQuery", kusto_query) \
            .option("accessToken", KDB_ACCESSTOKEN) \
            .load()
        return kustoDF
    except Exception as e:
        print(f"Error loading data from Kusto: {e}")
        raise







StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 37, Finished, Available, Finished)

In [20]:
def call_azureopenai(text):
    # Ensure the input is a list of messages
    if not isinstance(text, list):
        raise ValueError("Input text must be a list of messages")

    response = client.chat.completions.create(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME,
        messages=text,
        temperature=0
    )

    # Access the correct attribute for the generated text
    try:
        generated_text = response.choices[0].message.content
        return generated_text
    except AttributeError as e:
        print(f"Error accessing response content: {e}")
        print(f"Response structure: {response}")
        raise


StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 38, Finished, Available, Finished)

**Testing with questions** 

In [21]:
answers = 1
question = "What was the estimated volumetric water benefit from the replenishment projects contracted in FY23?"
answers_df = get_answer_kustodb(question, answers)

# Ensure 'content' is the correct column name in the DataFrame
answer = ""
try:
    for row in answers_df.rdd.toLocalIterator():
        answer += " " + row['content']
except Exception as e:
    print(f"Error processing DataFrame rows: {e}")
    raise

prompt = f'Question: {question}\nInformation: {answer}'

messages = [
    {"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer the following using the provided information and do not add anything else. Do not hallucinate"},
    {"role": "user", "content": prompt}
]

# Calling Azure OpenAI



result = call_azureopenai(messages)
display(result)

StatementMeta(, 480aa9fb-fac0-4f6e-9a9c-313e347a6bf9, 39, Finished, Available, Finished)

'The estimated volumetric water benefit from the replenishment projects contracted in FY23 was more than 25.4 million m³ over the lifetime of these projects.'

In [68]:
def answer_question(question, answers=1):
    # Retrieve answers from Kusto DB
    answers_df = get_answer_kustodb(question, answers)

    # Ensure 'content' is the correct column name in the DataFrame
    answer = ""
    try:
        for row in answers_df.rdd.toLocalIterator():
            answer += " " + row['content']
    except Exception as e:
        print(f"Error processing DataFrame rows: {e}")
        raise

    # Construct the prompt for Azure OpenAI
    prompt = f'Question: {question}\nInformation: {answer}'

    # Prepare the messages for Azure OpenAI
    messages = [
        {"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer the following using the provided information and do not add anything else. Do not hallucinate"},
        {"role": "user", "content": prompt}
    ]

    # Call Azure OpenAI to get the completion response
    result = call_azureopenai(messages)

    # Return the result
    return result

# Example usage
question = "What is the Total amount of solid waste diverted from landfills and incinerators in FY23?"
result = answer_question(question)
print(result)


StatementMeta(, 256818ca-a4a0-4a3e-9d94-7c6ad228cdfd, 94, Finished, Available, Finished)

The total amount of solid waste diverted from landfills and incinerators in FY23 is **18,537 metric tons**.


In [69]:
question = "What is the Total amount of solid waste diverted from landfills and incinerators in FY23?"
result = answer_question(question)
print(result)

StatementMeta(, 256818ca-a4a0-4a3e-9d94-7c6ad228cdfd, 95, Finished, Available, Finished)

The total amount of solid waste diverted from landfills and incinerators in FY23 is **18,537 metric tons**.
